<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Setup" data-toc-modified-id="Setup-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Setup</a></span><ul class="toc-item"><li><span><a href="#Check-your-sample-table-(i.e.-metadata-file)" data-toc-modified-id="Check-your-sample-table-(i.e.-metadata-file)-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Check your sample table (i.e. metadata file)</a></span></li><li><span><a href="#Check-your-TRN" data-toc-modified-id="Check-your-TRN-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Check your TRN</a></span></li><li><span><a href="#Load-the-data" data-toc-modified-id="Load-the-data-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Load the data</a></span></li></ul></li><li><span><a href="#Regulatory-iModulons" data-toc-modified-id="Regulatory-iModulons-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Regulatory iModulons</a></span></li><li><span><a href="#Check-for-single-gene-iModulons" data-toc-modified-id="Check-for-single-gene-iModulons-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Check for single gene iModulons</a></span></li><li><span><a href="#Explained-Variance" data-toc-modified-id="Explained-Variance-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Explained Variance</a></span></li><li><span><a href="#iModulon-Size" data-toc-modified-id="iModulon-Size-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>iModulon Size</a></span></li><li><span><a href="#Save-iModulon-object" data-toc-modified-id="Save-iModulon-object-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Save iModulon object</a></span></li></ul></div>

# Setup
This IPython notebook will walk through the steps of characterizing iModulons through semi-automated tools. You will need:
* M and A matrices
* Expression data (e.g. `log_tpm_norm.csv`)
* Gene table and KEGG/GO annotations (Generated in `gene_annotation.ipynb`)
* Sample table, with a column for `project` and `condition`
* TRN file

Optional:
* iModulon table (if you already have some characterized iModulons)

In [1]:
from pymodulon.core import IcaData
from pymodulon.io import save_to_json
from pymodulon.util import explained_variance
from os import path
import pandas as pd
import re
from Bio.KEGG import REST
from tqdm.notebook import tqdm

In [2]:
# Enter the location of your data here
data_dir = '../data/precise2/'

## Check your sample table (i.e. metadata file)
Your metadata file will probably have a lot of columns, most of which you may not care about. Feel free to save a secondary copy of your metadata file with only columns that seem relevant to you. The two most important columns are:
1. `project`
2. `condition`

Make sure that these columns exist in your metadata file

In [3]:
df_metadata = pd.read_csv(path.join(data_dir,'metadata_qc.csv'),index_col=0)
df_metadata[['project','condition']].head()

,project,condition
ecoli_00001,control,wt_glc
ecoli_00002,control,wt_glc
ecoli_00003,fur,wt_dpd
ecoli_00004,fur,wt_dpd
ecoli_00005,fur,wt_fe


In [4]:
print(df_metadata.project.notnull().all())
print(df_metadata.condition.notnull().all())

True
True


## Check your TRN

Each row of the TRN file represents a regulatory interaction.  
**Your TRN file must have the following columns:**
1. `regulator` - Name of regulator (`/` or `+` characters will be converted to `;`)
1. `gene_id` - Locus tag of gene being regulated

The following columns are optional, but are helpful to have:
1. `regulator_id` - Locus tag of regulator
1. `gene_name` - Name of gene (can automatically update this using `name2num`)
1. `direction` - Direction of regulation ('+' for activation, '-' for repression, '?' or NaN for unknown)
1. `evidence` - Evidence of regulation (e.g. ChIP-exo, qRT-PCR, SELEX, Motif search)
1. `PMID` - Reference for regulation

You may add any other columns that could help you. TRNs may be saved as either CSV or TSV files. See below for an example:

In [5]:
df_trn = pd.read_csv(path.join(data_dir,'TRN.csv'), index_col=0)
df_trn.head()

,regulator,gene_name,gene_id,effect,evidence,source
0,AccB,accB,b3255,-,0.0,RegulonDB_TF
1,MarA,ygiB,b3037,+,0.0,RegulonDB_TF
2,MarA,ygiC,b3038,+,0.0,RegulonDB_TF
3,MarA,yncE,b1452,+,0.0,RegulonDB_TF
4,MetJ,ahpC,b0605,-,0.0,RegulonDB_TF


The `regulator` and `gene_id` must be filled in for each row

In [6]:
print(df_trn.regulator.notnull().all())
print(df_trn.gene_id.notnull().all())

True
True


## Load the data
You're now ready to load your IcaData object!

In [5]:
ica_data = IcaData(M = path.join(data_dir,'M.csv'),
                   A = path.join(data_dir,'A.csv'),
                   X = path.join(data_dir,'log_tpm_norm_qc.csv'),
                   gene_table = path.join(data_dir,'gene_info.csv'),
                   sample_table = path.join(data_dir,'metadata_qc.csv'),
                   imodulon_table=path.join(data_dir,'imodulon_table.csv'),
                   trn = path.join(data_dir,'TRN.csv'),
                   log_tpm = path.join(data_dir, 'log_tpm_qc.csv'),
                  dagostino_cutoff=550)

/Users/cam/Projects/pymodulon/pymodulon/core.py:402: UserWarning: The following genes are in the TRN but not in your M matrix: {'b3978', 'b2396', 'b4616', 'b2815', 'b4592', 'b3119', 'b4164', 'b4457', 'b0202', 'b4417', 'b2856', 'b4624', 'b4640', 'b3046', 'b4438', 'b4618', 'b0536', 'b4718', 'b3758', 'b4663', 'b2591', 'b2849', 'b4445', 'b3855', 'b1348', 'b4666', 'b4625', 'b2691', 'b3976', 'b3853', 'b3489', 'b1470', 'b2401', 'b2652', 'b4443', 'b4665', 'b4609', 'b1986', 'b4442', 'b2911', 'b4436', 'b4691', 'b4762', 'b2915', 'b4667', 'b4585', 'b0371', 'b2339', 'b4687', 'b1989', 'b4427', 'b4586', 'b4370', 'b1569', 'b4450', 'b4444', 'b3851', 'b1977', 'b3971', 'b0303', 'b3274', 'b4699', 'b4513', 'b3278', 'b3171', 'b3797', 'b4458', 'b4803', 'b1666', 'b4010', 'b0971', 'b2967', 'b3277', 'b4669', 'b1510', 'b4368', 'b1665', 'b0673', 'b3759', 'b1576', 'b2018', 'b4763', 'b0364', 'b3272', 'b1365', 'b0743', 'b4741', 'b0204', 'b0883', 'b3889', 'b2598', 'b4719', 'b4456', 'b2621', 'b4432', 'b4451', 'b3969',

# Regulatory iModulons
Use `compute_trn_enrichment` to automatically check for Regulatory iModulons. The more complete your TRN, the more regulatory iModulons you'll find.

In [14]:
ica_data.compute_trn_enrichment(max_regs=2, evidence=[2], save=True)

,imodulon,regulator,pvalue,qvalue,precision,recall,f1score,TP,regulon_size,imodulon_size,n_regs
0,4,RpoD+CysB,9.446050e-13,3.573441e-08,0.538462,0.225806,0.318182,7.0,13.0,31.0,2.0
1,4,CysB,1.049301e-11,1.984752e-07,0.411765,0.225806,0.291667,7.0,17.0,31.0,1.0
2,5,RpoD+HprR,5.987020e-17,1.132445e-12,1.000000,0.545455,0.705882,6.0,6.0,11.0,2.0
3,5,RpoD+CusR,5.987020e-17,1.132445e-12,1.000000,0.545455,0.705882,6.0,6.0,11.0,2.0
4,5,HprR,4.186643e-16,2.262581e-12,0.857143,0.545455,0.666667,6.0,7.0,11.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
511,212,ArgR+NtrC,1.056311e-12,1.998012e-08,1.000000,0.263158,0.416667,5.0,5.0,19.0,2.0
512,212,ArgR+RpoN,1.056311e-12,1.998012e-08,1.000000,0.263158,0.416667,5.0,5.0,19.0,2.0
513,212,RpoS+NtrC,2.625181e-10,3.310353e-06,0.500000,0.263158,0.344828,5.0,10.0,19.0,2.0
514,212,RpoS+ArgR,8.204829e-10,7.759717e-06,0.416667,0.263158,0.322581,5.0,12.0,19.0,2.0


# Check for single gene iModulons

In [15]:
sg_imods = ica_data.find_single_gene_imodulons(save=True)

In [16]:
for sg_mod in sg_imods:
    sg_name = ica_data.view_imodulon(sg_mod).sort_values(
        by='gene_weight', ascending=False).iloc[0, :]['gene_name']
    ica_data.rename_imodulons({sg_mod:f'single_gene_{sg_name}'})

In [17]:
ica_data.imodulon_names[:5]

[0, 'single_gene_yzfA', 'single_gene_ytiD', 3, 4]

# Explained Variance

In [31]:
exp_vars = [explained_variance(ica_data, imodulons=[imod]) for imod in ica_data.imodulon_names]

In [32]:
ica_data.imodulon_table['exp_var'] = exp_vars

# iModulon Size

In [35]:
ica_data.imodulon_table['imodulon_size'] = [ica_data.view_imodulon(imod).shape[0]
                                            for imod in ica_data.imodulon_names]

# Save iModulon object

This will save your iModulon table, your thresholds, and any other information stored in the ica_data object.

In [ ]:
save_to_json(ica_data,'../data/precise2/precise2', compress=True)